# Classifier training

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import scipy.stats as sm
import sklearn

# Loading the data

In [2]:
names=['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']
df = pd.read_csv('data/out.csv')

df.head()

,age,sex,chest_pain,rest_blood_pressure,cholestorol,fast_blood_sugar,rest_ecg,max_heart_rate,exercise_angina,st_depression,st_slope,major_vessels,thal,target
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,1
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [3]:
y = df.iloc[:, -1].values
X = df.iloc[:, :-1]

## Split data
Splitting the data into two sets (`train` and `test`) means that the test set can be used as a new, unbiased dataset to evaluate the performance of the model against. This also helps detect the phenomenon of overfitting, where a model is extremely tailored to the training dataset, and therefore performs poorly on unseen datasets.

In this case, 80% of the samples will go to the training set, and the remaining 20 will go to the test set.

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print("X_train:", X_train.shape, "X_test:", X_test.shape)
print("y_train:", y_train.shape, "y_test:", y_test.shape)

X_train: (736, 13) X_test: (184, 13)
y_train: (736,) y_test: (184,)


## Scale data
Variables with much broader ranges of values might disproportionately effect the final model, so standardising the data ensures that all features are proportionately weighted. 

The formula used for feature standardisation transforms each feature to have zero-mean and unit-variance:

$ \large{x' = \frac{x - \overline{x}}{\sigma}} $

Where $x$ is the original feature vector, $ \bar{x}=average(x)$ is the mean of that feature vector, and $\sigma$ is its standard deviation.

In [5]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Training the classifiers
The following models will be compared:

Using `sklearn`: 
- Decision tree 
- Support Vector Machine
- Logistic Regression

Using `Tensorflow`:
- Artificial Neural Network

## Neural Network

The Neural Network was built with trial and error, until the best performing network was found. 
It contains:
- 13 input nodes, which correspond to the 13 input features
- 6 nodes in the first hidden layer, which uses the `ReLU` activation function and the `uniform` bias initialiser
- 13 nodes in the second hidden layer, which also uses the `ReLU` activation function
- 1 output layer (the probability of whether a patient is sick or not)
- Stochastic gradient descent (`sgd`) for calculating the loss function
- Mean squared error (`mean_squared_error`) algorithm for optimising the loss function

<img src="data/nn.png" width="600" />

In [6]:
import tensorflow as tf
from tensorflow import keras

classifier = keras.Sequential()

# adding the input layer (input_dim) and the first hidden layer
classifier.add(keras.layers.Dense(6, bias_initializer = 'uniform', activation = 'relu', input_dim = 13))

# adding the second hidden layer
classifier.add(keras.layers.Dense(13, activation = 'relu'))

# adding the output layer which contains 1 neuron 
# (this is a binary classification problem)
classifier.add(keras.layers.Dense(1, activation = 'sigmoid'))

In [7]:
# compiling the neural network
classifier.compile(optimizer = 'sgd', loss = 'mean_squared_error', metrics = ['accuracy'])

# fitting the model 
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
736/736 [==============================] - 0s 431us/step - loss: 0.2355 - acc: 0.5938
Epoch 2/100
736/736 [==============================] - 0s 92us/step - loss: 0.2193 - acc: 0.7024
Epoch 3/100
736/736 [==============================] - 0s 94us/step - loss: 0.2074 - acc: 0.7120
Epoch 4/100
736/736 [==============================] - 0s 96us/step - loss: 0.1985 - acc: 0.7255
Epoch 5/100
736/736 [==============================] - 0s 92us/step - loss: 0.1916 - acc: 0.7473
Epoch 6/100
736/736 [==============================] - 0s 108us/step - loss: 0.1860 - acc: 0.7486
Epoch 7/100
736/736 [==============================] - 0s 133us/step - loss: 0.1812 - acc: 0.7595
Epoch 8/100
736/736 [==============================] - 0s 119us/step - loss: 0.1771 - acc: 0.7704
Epoch 9/100
736/736 [==============================] - 0s 93us/step - loss: 0.1735 - acc: 0.7731
Epoch 10/100
736/736 [==============================] - 0s 97us/step - loss: 0.1703 - acc: 0.7772
Epoch 11/100
736/736 [===

736/736 [==============================] - 0s 108us/step - loss: 0.1368 - acc: 0.8179
Epoch 84/100
736/736 [==============================] - 0s 116us/step - loss: 0.1366 - acc: 0.8193
Epoch 85/100
736/736 [==============================] - 0s 83us/step - loss: 0.1365 - acc: 0.8179
Epoch 86/100
736/736 [==============================] - 0s 83us/step - loss: 0.1364 - acc: 0.8220
Epoch 87/100
736/736 [==============================] - 0s 83us/step - loss: 0.1362 - acc: 0.8220
Epoch 88/100
736/736 [==============================] - 0s 85us/step - loss: 0.1361 - acc: 0.8207
Epoch 89/100
736/736 [==============================] - 0s 107us/step - loss: 0.1360 - acc: 0.8220
Epoch 90/100
736/736 [==============================] - 0s 120us/step - loss: 0.1358 - acc: 0.8220
Epoch 91/100
736/736 [==============================] - 0s 94us/step - loss: 0.1357 - acc: 0.8220
Epoch 92/100
736/736 [==============================] - 0s 82us/step - loss: 0.1356 - acc: 0.8207
Epoch 93/100
736/736 [=======

In [8]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(classifier, ".")

Using TensorFlow backend.


In [10]:
y_pred = classifier.predict(X_test)
y_pred = y_pred.flatten().round()

In [11]:
y_actu = pd.Series(y_test, name='Actual')
y_pred = pd.Series(y_pred, name='Predicted')

df_confusion = pd.crosstab(y_actu, y_pred)
print(df_confusion)

Predicted  0.0  1.0
Actual             
0           56   25
1            7   96


In [12]:
accuracy = 100 * np.sum(y_test == y_pred) / len(y_test)
print("Test accuracy:", accuracy)

y_train_pred = classifier.predict(X_train).flatten().round()
accuracy_train = 100 * np.sum(y_train == y_train_pred) / len(y_train)
print("Train accuracy:", accuracy_train)

Test accuracy: 82.6086956521739
Train accuracy: 82.47282608695652


## Decision Tree classifier

In [13]:
def get_accuracy():
    print("Sklearn accuracy:", sklearn.metrics.accuracy_score(y_test, y_pred))
    print("Sklearn recall:", sklearn.metrics.recall_score(y_test, y_pred))
    print("\nConfusion matrix (predicted vs actual)")
    print(pd.DataFrame(np.c_[sklearn.metrics.confusion_matrix(y_test, y_pred)]))  

In [14]:
from sklearn.tree import DecisionTreeClassifier  

# Create a decision tree Classifier
classifier = DecisionTreeClassifier()  

# Fit the model
classifier.fit(X_train, y_train)  

# Predict the response for test dataset
y_pred = classifier.predict(X_test)  

# Get accuracy results
get_accuracy()

Sklearn accuracy: 0.7336956521739131
Sklearn recall: 0.8252427184466019

Confusion matrix (predicted vs actual)
    0   1
0  50  31
1  18  85


## Support Vector Machine

In [18]:
from sklearn.svm import SVC

# Create an SVM Classifier
classifier = SVC(kernel='linear') # Linear Kernel

# Fit the model
classifier.fit(X_train, y_train)  

# Train the model and predict against the validation set
y_pred = classifier.predict(X_test)  

# Get accuracy results
get_accuracy()

Sklearn accuracy: 0.8260869565217391
Sklearn recall: 0.9223300970873787

Confusion matrix (predicted vs actual)
    0   1
0  57  24
1   8  95


## Logistic regression

In [19]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')

# Fit the model
classifier.fit(X_train, y_train)  

# Predict the response for test dataset
y_pred = classifier.predict(X_test)  

# Get accuracy results
get_accuracy()

Sklearn accuracy: 0.8315217391304348
Sklearn recall: 0.9223300970873787

Confusion matrix (predicted vs actual)
    0   1
0  58  23
1   8  95
